# Modal Symbolic Learning: Day 1

## Propositional Logic

### Formulas & Interpretations

In [42]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.update()
using SoleLogics
Pkg.status()

  Activating project at `~/Desktop/modal-symbolic-learning-course`
    Updating registry at `~/.julia/registries/General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
  No Changes to `~/Desktop/modal-symbolic-learning-course/Project.toml`
  No Changes to `~/Desktop/modal-symbolic-learning-course/Manifest.toml`


Status `~/Desktop/modal-symbolic-learning-course/Project.toml`
  [a93c6f00] DataFrames v1.6.1
  [7806a523] DecisionTree v0.12.4
  [7073ff75] IJulia v1.24.2
⌅ [add582a8] MLJ v0.19.5
  [c6f25543] MLJDecisionTreeInterface v0.4.0
  [e54bda2e] ModalDecisionTrees v0.3.1
  [91a5bcdd] Plots v1.39.0
  [7b3b3b3f] Sole v0.3.0
  [b002da8f] SoleLogics v0.6.8
  [4249d9c7] SoleModels v0.5.1
  [2913bbd2] StatsBase v0.34.2
  [9a3f8284] Random
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [2]:
p = Atom("it's alive")

Atom{String}: it's alive

In [3]:
q = Atom("it's mortal!")

Atom{String}: it's mortal!

In [4]:
φ = p ∧ q

SyntaxBranch{NamedConnective{:∧}}: it's alive ∧ it's mortal!

In [5]:
φ isa Formula && p isa Formula

true

In [6]:
# Create a method for negating any formula
function negateformula(f::Formula)
    return ¬f
end

negateformula (generic function with 1 method)

In [7]:
negateformula(φ)

SyntaxBranch{NamedConnective{:¬}}: ¬(it's alive ∧ it's mortal!)

In [8]:
negateformula(p)

SyntaxBranch{NamedConnective{:¬}}: ¬it's alive

In [9]:
# Obtain the string representation of a Formula
syntaxstring(φ)

"it's alive ∧ it's mortal!"

In [10]:
# I can also parse Formula's from standard string representations
φ = parseformula("it's alive ∧ it's mortal!")

SyntaxBranch{NamedConnective{:∧}}: it's alive ∧ it's mortal!

In [11]:
function my_own_string_representation(f::SyntaxBranch{NamedConnective{:¬}})
    return "It is not the case that $(my_own_string_representation(first(children(f))))"
end
function my_own_string_representation(f::SyntaxBranch{NamedConnective{:∧}})
    subformula1, subformula2 = children(f)
    return "both $(my_own_string_representation(subformula1)) and $(my_own_string_representation(subformula2))"
end
function my_own_string_representation(f::SyntaxLeaf)
    return syntaxstring(f)
end

my_own_string_representation (generic function with 3 methods)

In [12]:
my_own_string_representation(¬φ)

"It is not the case that both it's alive and it's mortal!"

In [13]:
Guido = TruthDict([p => true, q => false])

TruthDict with values:
┌──────────────┬────────────┐
│ it's mortal! │ it's alive │
│       String │     String │
├──────────────┼────────────┤
│            ⊥ │          ⊤ │
└──────────────┴────────────┘


In [14]:
Giovanni = TruthDict([p => true, q => true])

TruthDict with values:
┌──────────────┬────────────┐
│ it's mortal! │ it's alive │
│       String │     String │
├──────────────┼────────────┤
│            ⊤ │          ⊤ │
└──────────────┴────────────┘


In [15]:
check(p, Guido) # Guido is alive

true

In [16]:
check(φ, Guido) # But not both alive and mortal

false

In [17]:
check(φ, Giovanni) # Giovanni is both alive and mortal!

true

In [18]:
# These objects can actually be used as dictionaries from Formula to Truth values
# For example, we both assign ⊤ (top) to the atom "alive"
Guido[p], Giovanni[p]

(⊤, ⊤)

In [19]:
# What is ⊤?
⊤ isa Truth      &&     ⊤ isa Formula

true

In [20]:
# Now it's time to say it: these Unicode character are
#  not too comfy for a Jupyter Notebook. Let's use their aliases.
TOP, BOT, CONJUNCTION, DISJUNCTION, IMPLICATION
#  (but try typing \top<tab> in the Julia REPL)

(⊤, ⊥, ∧, ∨, →)

In [21]:
# Actually, indexing (with []) can be used to check generic Formula's
Guido[φ]

⊥

In [22]:
# This is syntactic sugar for the *interpretation* algorithm, which is actually more general than check!
interpret(φ, Guido)

⊥

In [23]:
# In fact, it also works under incomplete information.
# Notice how in this example, with an *unknown atom*, it uses the *known* information to simplify the formula
Giovanni[φ ∨ Atom("?Unknown property?")]

⊤

In [24]:
# So ultimately, `check` is just a shortcut for making sure that `interpret` simplifies the formula to ⊤
check(φ, Guido) == istop(interpret(φ, Guido))

true

In [25]:
# Let's generate random formulas
treeheight = 3
@atoms a b
φ2 = randformula(treeheight, [a,b], SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES)

SyntaxBranch{NamedConnective{:→}}: (a ∧ a ∨ (b → b)) → (¬(a → b))

In [26]:
# Let's control reproducibility, though ;)
using Random
rng = Random.MersenneTwister(1)
φ2 = randformula(rng, treeheight, [a,b], SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES)

SyntaxBranch{NamedConnective{:¬}}: ¬(b ∧ a ∧ ¬b)

In [27]:
normalize(φ2)

SyntaxBranch{NamedConnective{:→}}: (b ∧ a) → b

#### HOMEWORK EXERCISE 1
Check many, randomly-generated formulas on the alphabet ${p, q}$ on both `Guido` and `Giovanni`.
Do `Guido` and `Giovanni` have the same probability of satisfying a generic formula? Can you estimate this probability?

### Scalar logisets

Now, let's consider a propositional interpretation on scalar variables, and check formulas on an alphabet $\mathcal{A} \subseteq \{V < v, v \in \mathbb{R}\}$ on it.

In [28]:
import SoleLogics: syntaxstring

struct ConditionOnVariable
    i_variable::Integer
    threshold::Number
end

function syntaxstring(c::ConditionOnVariable; kwargs...)
    "V$(c.i_variable) < $(c.threshold)"
end

syntaxstring(ConditionOnVariable(2, 10))

"V2 < 10"

In [29]:
using SoleLogics: AbstractAssignment # Propositional Interpretations

struct TabularInterpretation{T<:Real} <: AbstractAssignment
    vals::Vector{T}
end

import SoleLogics: interpret

function interpret(a::Atom{ConditionOnVariable}, I::TabularInterpretation)
    cond = value(a)
    return (I.vals[cond.i_variable] < cond.threshold ? TOP : BOT)
end

interpret (generic function with 12 methods)

In [30]:
rng = Random.MersenneTwister(1)
n_variables = 4

I = TabularInterpretation(rand(rng, n_variables))

TabularInterpretation{Float64}([0.23603334566204692, 0.34651701419196046, 0.3127069683360675, 0.00790928339056074])

In [31]:
A = Atom.([ConditionOnVariable(v, t) for v in 1:n_variables for t in 0:0.1:1.0])
syntaxstring.(A)

44-element Vector{String}:
 "V1 < 0.0"
 "V1 < 0.1"
 "V1 < 0.2"
 "V1 < 0.3"
 "V1 < 0.4"
 "V1 < 0.5"
 "V1 < 0.6"
 "V1 < 0.7"
 "V1 < 0.8"
 "V1 < 0.9"
 "V1 < 1.0"
 "V2 < 0.0"
 "V2 < 0.1"
 ⋮
 "V3 < 1.0"
 "V4 < 0.0"
 "V4 < 0.1"
 "V4 < 0.2"
 "V4 < 0.3"
 "V4 < 0.4"
 "V4 < 0.5"
 "V4 < 0.6"
 "V4 < 0.7"
 "V4 < 0.8"
 "V4 < 0.9"
 "V4 < 1.0"

In [32]:
[interpret(cond, I) for cond in A]

44-element Vector{BooleanTruth}:
 ⊥
 ⊥
 ⊥
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊥
 ⊥
 ⋮
 ⊤
 ⊥
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤
 ⊤

In [33]:
rng = Random.MersenneTwister(32)

[begin
f = randformula(rng, 3, A, SoleLogics.BASE_PROPOSITIONAL_CONNECTIVES)
syntaxstring(f) => interpret(f, I)
end for _ in 1:10]

10-element Vector{Pair{String}}:
                 "(¬V2 < 0.4 ∧ (V1 < 0.5 → V4 < 0.8)) → (V4 < 0.4 ∧ V4 < 0.9 ∧ V2 < 0.9 ∧ V1 < 0.2)" => ⊤
                     "V4 < 0.1 ∨ V2 < 0.6 ∨ V4 < 0.8 ∨ V2 < 0.5 ∨ ¬V4 < 0.8 ∧ (V2 < 0.4 → V1 < 0.5)" => ⊤
                                                   "¬(V3 < 0.6 ∧ V4 < 0.9) ∧ ¬(V2 < 0.3 → V1 < 1.0)" => ⊥
                                                "¬¬V4 < 0.6 ∧ ((¬V1 < 0.0) → (V2 < 0.4 ∧ V2 < 0.3))" => ⊥
   "((V4 < 0.4 → V4 < 1.0) ∨ V1 < 0.7 ∨ V2 < 0.7) → ((V3 < 0.3 → V4 < 0.3) ∧ (V2 < 0.2 ∨ V3 < 0.5))" => ⊤
 "((V4 < 0.0 → V2 < 0.3) → (V4 < 0.2 ∧ V3 < 0.9)) ∨ ((V1 < 0.8 → V1 < 0.4) → (V2 < 0.7 → V2 < 0.0))" => ⊤
                           "¬V3 < 0.6 ∧ ¬V3 < 1.0 ∧ ((V3 < 0.5 → V3 < 0.5) → (V3 < 0.8 ∧ V4 < 0.3))" => ⊥
                                                                "¬(V3 < 0.6 ∧ V1 < 0.1 ∧ ¬V1 < 0.7)" => ⊤
                                                   "¬(V1 < 0.0 ∨ V3 < 0.5) ∧ ¬(V4 < 0.2 ∨ V1 < 0.8)" => ⊥
             

#### HOMEWORK EXERCISE 2
Check many, randomly-generated formulas on many, randomly-generated tabular interpretations,
and store the formulas that satisfy the highest number of instances!

## Modal Logic

In [34]:
# Instantiate a frame with 5 worlds and 5 edges
using Graphs

worlds = SoleLogics.World.(1:5)
edges = Edge.([(1,2), (1,3), (2,4), (3,4), (3,5)])
fr = SoleLogics.ExplicitCrispUniModalFrame(worlds, Graphs.SimpleDiGraph(edges))

SoleLogics.ExplicitCrispUniModalFrame{SoleLogics.World{Int64}, SimpleDiGraph{Int64}} with
- worlds = ["1", "2", "3", "4", "5"]
- accessibles = 
	1 -> [2, 3]
	2 -> [4]
	3 -> [4, 5]
	4 -> []
	5 -> []


In [35]:
# Pick the first world
w1 = worlds[1]

# Enumerate the world that are accessible from the first world
accessibles(fr, w1)

2-element Vector{SoleLogics.World{Int64}}:
 SoleLogics.World{Int64}(2)
 SoleLogics.World{Int64}(3)

In [36]:
# That's an iterator of worlds... If I want to see them, I'll collect them.
collect(accessibles(fr, w1))

2-element Vector{SoleLogics.World{Int64}}:
 SoleLogics.World{Int64}(2)
 SoleLogics.World{Int64}(3)

In [37]:
# Assign each world a propositional interpretation
valuation = Dict([
        worlds[1] => TruthDict([p => TOP, q => BOT]),
        worlds[2] => TruthDict([p => TOP, q => TOP]),
        worlds[3] => TruthDict([p => TOP, q => BOT]),
        worlds[4] => TruthDict([p => BOT, q => BOT]),
        worlds[5] => TruthDict([p => BOT, q => TOP]),
     ])

# Instantiate a Kripke structure
K = KripkeStructure(fr, valuation)

KripkeStructure{SoleLogics.ExplicitCrispUniModalFrame{SoleLogics.World{Int64}, SimpleDiGraph{Int64}}, Dict{SoleLogics.World{Int64}, TruthDict{Dict{Atom{String}, BooleanTruth}}}} with
- frame = SoleLogics.ExplicitCrispUniModalFrame{SoleLogics.World{Int64}, SimpleDiGraph{Int64}} with
- worlds = ["1", "2", "3", "4", "5"]
- accessibles = 
	1 -> [2, 3]
	2 -> [4]
	3 -> [4, 5]
	4 -> []
	5 -> []
- valuations = 
	1 -> TruthDict([it's mortal! => ⊥, it's alive => ⊤])
	2 -> TruthDict([it's mortal! => ⊤, it's alive => ⊤])
	3 -> TruthDict([it's mortal! => ⊥, it's alive => ⊤])
	4 -> TruthDict([it's mortal! => ⊥, it's alive => ⊥])
	5 -> TruthDict([it's mortal! => ⊤, it's alive => ⊥])


In [38]:
# Generate a random modal formula
φmodal = randformula(Random.MersenneTwister(14), 3, [p,q], SoleLogics.BASE_MODAL_CONNECTIVES)

println(syntaxstring(φmodal))

# Check the formula on each world of the Kripke structure
[w => check(φmodal, K, w) for w in worlds]

□(it's alive → it's mortal!) ∨ ¬(it's alive → it's alive)


5-element Vector{Pair{SoleLogics.World{Int64}, Bool}}:
 SoleLogics.World{Int64}(1) => 0
 SoleLogics.World{Int64}(2) => 1
 SoleLogics.World{Int64}(3) => 1
 SoleLogics.World{Int64}(4) => 1
 SoleLogics.World{Int64}(5) => 1

#### EXERCISE 3
Define a structure for representing a *modal* interpretation on scalar variables.

In [39]:
using SoleLogics: AbstractFrame, World, AbstractKripkeStructure

# TODO:
# struct ModalInterpretation{FR<:AbstractFrame,T<:Real} <: AbstractKripkeStructure
#     frame::FR
#     vals::???
# end

import SoleLogics: interpret, frame

# Retrieve the interpretation's frame
frame(i::ModalInterpretation) = i.frame

# TODO:
# function interpret(a::Atom{ConditionOnVariable}, I::ModalInterpretation, w::World)
#     cond = value(a)
#     v = ???
#     return (v < cond.threshold ? TOP : BOT)
# end

LoadError: UndefVarError: `ModalInterpretation` not defined

In [40]:
rng = Random.MersenneTwister(1)
n_variables = 4
n_worlds = 5
n_edges = 7
n_formulas = 10

worlds = SoleLogics.World.(1:n_worlds)
g = SimpleDiGraph(n_worlds, n_edges; rng)
fr = SoleLogics.ExplicitCrispUniModalFrame(worlds, g)
variable_values = [rand(n_variables) for w in worlds]

Imodal = ModalInterpretation(fr, variable_values)

for i_formula in 1:n_formulas
    φmodal = randformula(Random.MersenneTwister(i_formula), 2, A, SoleLogics.BASE_MODAL_CONNECTIVES)

    println(syntaxstring(φmodal) => ["w$(SoleLogics.name(w))" => check(φmodal, Imodal, w) for w in worlds])
    println()
end

LoadError: UndefVarError: `ModalInterpretation` not defined

#### EXERCISE 4
Check many, randomly-generated *modal* formulas on many, randomly-generated *modal* interpretations,
and store the formulas that satisfy the highest number of instances!

In [41]:
# TODO